In [2]:
! pip install pandas
! pip install pydicom
! pip install seaborn

import glob, pylab, pandas as pd
import pydicom, numpy as np
from torch.utils.data import DataLoader, TensorDataset
from skimage.transform import resize
from skimage.exposure import equalize_hist


import matplotlib
import matplotlib.pyplot as plt
from IPython.display import Image, display, clear_output
import numpy as np
%matplotlib nbagg
%matplotlib inline
import seaborn as sns
sns.set_style("whitegrid")
sns.set_palette(sns.dark_palette("purple"))

import torch
cuda = torch.cuda.is_available()

from torch.utils.data.sampler import SubsetRandomSampler
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor
from functools import reduce

import torch.nn as nn
from torch.nn.functional import softplus
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim
from torch.nn import Linear, GRU, Conv2d, Dropout, Dropout2d, MaxPool2d, BatchNorm1d, BatchNorm2d, ReLU, ELU,ConvTranspose2d, MaxUnpool2d
from torch.nn.functional import relu, elu, relu6, sigmoid, tanh, softmax, dropout, dropout2d
import time

In [92]:
# Define size variables
IMG_SIZE = 224
NUM_CONV = 4
height = IMG_SIZE
width = IMG_SIZE
channels = 1
num_features = 224**2

# Regulization
L2_reg = 1e-6
do_p = 0.2
# Conv Layers
conv_out_channels = [8, 16, 32, 64]
conv_kernel = [14, 11, 5, 5]
conv_padding = [7, 5, 2, 2]
conv_stride = [2, 2, 1, 1]

# MaxPool Layers
pool_kernel = 4
pool_padding = 2
pool_stride = 2

# Calculating the dimensions 
def compute_conv_dim(height, width, kernel_size, padding_size, stride_size):
    height_new = int((height - kernel_size + 2 * padding_size) / stride_size + 1)
    width_new =  int((width  - kernel_size + 2 * padding_size) / stride_size + 1)
    return [height_new, width_new]

def compute_final_dimension(height, width, last_num_channels, num_layers):
    # First conv layer
    CNN_height = height
    CNN_width = width
    for i in range(num_layers):
        # conv layer
        CNN_height, CNN_width = compute_conv_dim(CNN_height, CNN_width, conv_kernel[i], conv_padding[i], conv_stride[i])
        # maxpool layer
        CNN_height, CNN_width = compute_conv_dim(CNN_height, CNN_width, pool_kernel, pool_padding, pool_stride)
    final_dim = CNN_height * CNN_width * last_num_channels
    print(CNN_height,CNN_width)
    return [final_dim, CNN_height, CNN_width]
    
######## Image has to be: (num, channels, height, width)!!!! #########
class CNN_VAE(nn.Module):
    
    def __init__(self, latent_features, num_samples):
        super(CNN_VAE, self).__init__()
        
        self.latent_features = latent_features
        self.num_samples = num_samples
        
        # Calculate final size of the CNN
        self.final_dim = compute_final_dimension(height,width,conv_out_channels[-1],4)
        ## CNN encoder
        self.conv_layers = []
        self.batchnorm = []
        self.indices = []
        input_channels = channels
        for i in range(NUM_CONV):
            self.conv_layers.append(Conv2d( in_channels=input_channels,
                                            out_channels=conv_out_channels[i],
                                            kernel_size=conv_kernel[i],
                                            stride=conv_stride[i],
                                            padding=conv_padding[i]))
            self.batchnorm.append(BatchNorm2d(conv_out_channels[i]))
            input_channels = conv_out_channels[i]
        self.relu = ReLU();
        self.dropout = Dropout2d(p=do_p)
        self.maxPool_layers = MaxPool2d(  kernel_size=pool_kernel, 
                                        stride=pool_stride,
                                        padding=pool_padding,
                                        return_indices = True)
        
        self.CNN_to_latent = Linear(in_features=self.final_dim[0], out_features=self.latent_features*2)
        self.latent_to_CNN = Linear(in_features=self.latent_features, out_features=self.final_dim[0])
        # The latent code must be decoded into the original image
        self.deconv_layers = []
        self.batchnorm_deconv = []
        for i in reversed(range(NUM_CONV)):
            if i == 0:
                output_channels = channels*2
            else:
                output_channels = conv_out_channels[i-1]
                
            self.deconv_layers.append(ConvTranspose2d(  in_channels=conv_out_channels[i],
                                                        out_channels=output_channels,
                                                        kernel_size=conv_kernel[i],
                                                        stride=conv_stride[i],
                                                        padding=conv_padding[i]))
            self.batchnorm_deconv.append(BatchNorm2d(output_channels))
        
        self.maxunpool = MaxUnpool2d(   kernel_size=pool_kernel, 
                                        stride=pool_stride,
                                        padding=pool_padding)       
        

    def forward(self, x): 
        outputs = {}
        self.indices = []
        self.layer_size = []
        for i in range(NUM_CONV):
            print("Shape of x after ",i,"'th layer: ",x.shape)
            x = self.conv_layers[i]((x))
            self.layer_size.append(x.shape[-1])
            x, tmp_indices = self.maxPool_layers(x) 
            self.indices.append(tmp_indices)
            x = self.relu(x)
            x = self.batchnorm[i]((x))
            x = self.dropout(x)
        print("Shape of x after ",i+1,"'th layer: ",x.shape)
        
        #x = self.encoder(x)
        if torch.sum(torch.isnan(x))>0:
            print('output from encoder is NaN')
            print(torch.sum(torch.isnan(x)))

        batch_size = x.size(0)
        x = x.view( batch_size, -1)
        # x = x.view(num_samples,-1) # fold out the CNN layers
        x = self.CNN_to_latent(x)
        if torch.sum(torch.isnan(x))>0:
            print("After fully connected layer:")
            print(torch.sum(torch.isnan(x)))
     
        # Split encoder outputs into a mean and variance vector
        mu, log_var = torch.chunk(x, 2, dim=-1)
        
        # Make sure that the log variance is positive
        log_var = softplus(log_var)
        
        # :- Reparametrisation trick
        # a sample from N(mu, sigma) is mu + sigma * epsilon
        # where epsilon ~ N(0, 1)
                
        # Don't propagate gradients through randomness
        with torch.no_grad():
            batch_size = mu.size(0)
            epsilon = torch.randn(batch_size, self.num_samples, self.latent_features)
            
            if cuda:
                epsilon = epsilon.cuda()
        
        sigma = torch.exp(log_var/2)
        
        # We will need to unsqueeze to turn (batch_size, latent_dim) -> (batch_size, 1, latent_dim)
        z = mu.unsqueeze(1) + epsilon * sigma.unsqueeze(1) 
        if torch.sum(torch.isnan(z))>0:
            print("z:")
            print(torch.sum(torch.isnan(z)))
                  
        print("z.shape before view", z.shape)
        x = z.view(-1,latent_features)
        print("x.shape after view", x.shape)
        x = self.latent_to_CNN(x)
        print("x.shape after latent2CNN", x.shape)
        x = x.view(batch_size*num_samples, conv_out_channels[NUM_CONV-1], self.final_dim[1], self.final_dim[2])
        print("x.shape input to decoder:", x.shape)
         # Run through decoder
        print(self.indices[1].shape)
        for i in range(NUM_CONV):
            print("Shape of x after ", i,"'th layer: ",x.shape)
            x = self.maxunpool(x,self.indices[NUM_CONV-1-i].repeat(num_samples,1,1,1),output_size=[self.layer_size[NUM_CONV-1-i],self.layer_size[NUM_CONV-1-i]]) 
            x = self.deconv_layers[i]((x))
            x = self.relu(x)
            x = self.batchnorm_deconv[i]((x))
            x = self.dropout(x)
        print("Shape of x after ", i+1,"'th layer: ",x.shape)    
        x = x.view(batch_size,num_samples*2,height,width)
        
        print("Shape of x after deconvolution: ",x.shape)    
        if torch.sum(torch.isnan(x))>0:
            print("After decoder:")
            print(torch.sum(torch.isnan(x)))

        x_mean, x_log_var = torch.chunk(x, 2, dim=1) # the mean and log_var reconstructions from the decoder
        
        # The original digits are on the scale [0, 1]
        x_hat = torch.sigmoid(x_mean) # to scale for showing an image
        x_log_var = softplus(x_log_var)
        
        # Mean over samples
        x_hat = torch.mean(x_hat, dim=1)
        x_log_var= torch.mean(x_log_var, dim=1)
        
        # Resize x_hat from [batch_size, no_features] to [batch_size, channels, height, width]
        x_hat = x_hat.view( batch_size, 1, height, width)
        x_log_var = x_log_var.view( batch_size, 1, height, width)
        
        outputs["x_hat"] = x_hat # This is used for visulizations only 
        outputs["z"] = z
        outputs["mu"] = mu
        outputs["log_var"] = log_var
        
        # image recontructions (notice they are outputted as matrices)
        outputs["x_mean"] = x_hat #torch.reshape(x_mean,(-1,height,width)) # mean reconstructions (for loss!!!)
        outputs["x_log_var"] = x_log_var #torch.reshape(x_log_var,(-1,height,width)) # log var reconstructions (for loss!!!)
        
        return outputs


latent_features = 4
# The number of samples used then initialising the VAE, 
# is number of samples drawn from the distribution
num_samples = 10

net = CNN_VAE(latent_features, num_samples)
#print(net)
# Transfer model to GPU if available
if cuda:
    net = net.cuda()
x = torch.randn(32, 1, 224,224)
y = net.forward(x)

5 5
Shape of x after  0 'th layer:  torch.Size([32, 1, 224, 224])
Shape of x after  1 'th layer:  torch.Size([32, 8, 57, 57])
Shape of x after  2 'th layer:  torch.Size([32, 16, 15, 15])
Shape of x after  3 'th layer:  torch.Size([32, 32, 8, 8])
Shape of x after  4 'th layer:  torch.Size([32, 64, 5, 5])
z.shape before view torch.Size([32, 10, 4])
x.shape after view torch.Size([320, 4])
x.shape after latent2CNN torch.Size([320, 1600])
x.shape input to decoder: torch.Size([320, 64, 5, 5])
torch.Size([32, 16, 15, 15])
Shape of x after  0 'th layer:  torch.Size([320, 64, 5, 5])
Shape of x after  1 'th layer:  torch.Size([320, 32, 8, 8])
Shape of x after  2 'th layer:  torch.Size([320, 16, 15, 15])
Shape of x after  3 'th layer:  torch.Size([320, 8, 57, 57])
Shape of x after  4 'th layer:  torch.Size([320, 2, 224, 224])
Shape of x after deconvolution:  torch.Size([32, 20, 224, 224])


In [81]:
x = torch.randn(1, 2)
print(x)
x = x.repeat(10,1)
print(x)


tensor([[0.0510, 0.0154]])
tensor([[0.0510, 0.0154],
        [0.0510, 0.0154],
        [0.0510, 0.0154],
        [0.0510, 0.0154],
        [0.0510, 0.0154],
        [0.0510, 0.0154],
        [0.0510, 0.0154],
        [0.0510, 0.0154],
        [0.0510, 0.0154],
        [0.0510, 0.0154]])
